- Update: 2023.04.23

# 0. 配置参数

- `url`：Summit.com

In [1]:
url = 'https://ecat.spectrapremium.com/en/parts?line=oil_pans&sort=part-number&limit=100&page='

chromedriver = '../../chromedriver.exe'

In [2]:
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time

In [3]:
chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'

driver = Chrome(service=Service(chromedriver), options=chrome_options)
driver.maximize_window()
time.sleep(0.3)

In [4]:
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
list_url = []

In [ ]:
for i in range(6):
    print(i)
    driver.get(url + str(i+1))
    
    list_item = driver.find_elements(By.XPATH, '//*[@class="tuile-produit__code-produit"]')
    for item in list_item:
        list_url.append('https://ecat.spectrapremium.com/en/parts/' + item.text)

In [ ]:
pd.DataFrame({'Product': list_url}).to_excel('./menu-LS_Oil_Pan.xlsx', index=False)
print('总数量：' + str(len(list_url)))
print()

# 1. 爬取

In [5]:
list_product = pd.read_excel('./menu-LS_Oil_Pan.xlsx')['Product'].to_list()

list_product

['https://ecat.spectrapremium.com/en/parts/BMP02A',
 'https://ecat.spectrapremium.com/en/parts/BMP03A',
 'https://ecat.spectrapremium.com/en/parts/BMP04A',
 'https://ecat.spectrapremium.com/en/parts/BMP05A',
 'https://ecat.spectrapremium.com/en/parts/BMP06A',
 'https://ecat.spectrapremium.com/en/parts/BMP07A',
 'https://ecat.spectrapremium.com/en/parts/BMP08A',
 'https://ecat.spectrapremium.com/en/parts/BMP09A',
 'https://ecat.spectrapremium.com/en/parts/BMP10A',
 'https://ecat.spectrapremium.com/en/parts/BMP11A',
 'https://ecat.spectrapremium.com/en/parts/BMP12A',
 'https://ecat.spectrapremium.com/en/parts/BMP13A',
 'https://ecat.spectrapremium.com/en/parts/BMP14A',
 'https://ecat.spectrapremium.com/en/parts/BMP15A',
 'https://ecat.spectrapremium.com/en/parts/BMP16A',
 'https://ecat.spectrapremium.com/en/parts/BMP17A',
 'https://ecat.spectrapremium.com/en/parts/BMP18A',
 'https://ecat.spectrapremium.com/en/parts/BMP19A',
 'https://ecat.spectrapremium.com/en/parts/BMP20A',
 'https://ec

In [6]:
df = pd.DataFrame(columns=['Item',
                           '图片',
                           '链接',
                           '图片链接',
                           'status'])

In [7]:
i = 0
driver.get(list_product[i])
driver.maximize_window()
time.sleep(0.3)

In [8]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="part__titre-code"]')))
item = driver.find_element(By.XPATH, '//*[@class="part__titre-code"]').text
WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="mz-figure mz-hover-zoom"]/img')))
pic = driver.find_element(By.XPATH, '//*[@class="mz-figure mz-hover-zoom"]/img').get_attribute('src')
vehicle = ''
list_row = driver.find_elements(By.XPATH, '//*[@class="part__titre-groupe-vehicules"]')
for row in list_row:
    vehicle += row.text + '\n'

print(item + '; ' + pic + '; ' + vehicle)

BMP02A; https://ecat.spectrapremium.com/img/produits/imagettes/BMP02A_TOP_P04_220.jpg; BMW 318i Base L4 1.8L
BMW 318is Base L4 1.8L



In [10]:
df_temp = pd.DataFrame({'Item': [str(item)],
                        '图片': [''],
                        '链接': [str(list_product[i])],
                        '图片链接': [str(pic)],
                        'status': ['ok']})

In [11]:
WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="part__nom-spec-valeur-spec"]')))
list_specification = driver.find_elements(By.XPATH, '//*[@class="part__nom-spec-valeur-spec"]')

for specification in list_specification:
    key = specification.find_element(By.XPATH, './*[@class="part__nom-spec"]').text
    value = specification.find_element(By.XPATH, './*[@class="part__valeur-spec"]').text
    df_temp[key.lower()] = value
    
df_temp

,Item,图片,车型,链接,图片链接,status,DESCRIPTION,MATERIAL,HEIGHT,DRAIN PLUG THREAD,POP. CODE
0,BMP02A,,BMW 318i Base L4 1.8L\nBMW 318is Base L4 1.8L\n,https://ecat.spectrapremium.com/en/parts/BMP02A,https://ecat.spectrapremium.com/img/produits/i...,ok,Automotive Engine Oil Pan,Aluminum,3.25 in.,M12 X 1.25,D


In [ ]:
df = pd.concat([df, df_temp], ignore_index=True).fillna('')

driver.close()

df